In [19]:
from __future__ import print_function

In [3]:
#!jupyter nbconvert --to python Dataset.ipynb
from Dataset import GenericDataset

In [4]:
import cv2
import random
import torch
import torchvision.transforms as transforms
import torch.utils.data as data

In [23]:
import glob
import os
import shutil

In [5]:
class videoFrameError(Exception):
    """
        Raise when reading a non-existing frame in a video
        Attribute : 
            videoName --- video name or path
            frame     --- frame number
    """
    def __init__(self, videoName, frame):
        self.videoName = videoName
        self.frame = frame
    def __str__(self):
        return repr(self.videoName) + repr(self.frame)

In [6]:
def ReadRandomSequence(videoName, nbFrame, transform=transforms.ToTensor(), dropFrame=1):
    """
        Read a nbFrame number of frame randomly in the video
    """
    cap = cv2.VideoCapture(videoName)
    if cv2.CAP_PROP_FRAME_COUNT - (nbFrame*dropFrame) < 0:
        firstFrame=cv2.CAP_PROP_FRAME_COUNT
    else:
        firstFrame = random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT)- (nbFrame*dropFrame))
    
    ret = cap.set(cv2.CAP_PROP_POS_FRAMES,firstFrame) #go to first frame
    cframe = firstFrame #current frame

    if not ret:
        raise videoFrameError(videoName, cframe)
    t = torch.Tensor(nbFrame,3,225,225)
    for i in range(nbFrame):
        ret, frame = cap.read()
        cframe += 1
        if not ret:
            raise videoFrameError(videoName, cframe)    
        t[i] = transform(frame)
    return t

In [40]:
def ReadNFrame(videoName, nbFrame, transform=transforms.ToTensor(), dropFrame=1):
    """
        Read a nbFrame frame and concatenate them
        
        Input : videoName: path to video
                nbFrame : number of frame to read
                transform : transform to apply to each frame
                dropFrame : number of frame to drop between each frame
        
    """
    cap = cv2.VideoCapture(videoName)
    
    #verify video length
    if cap.get(cv2.CAP_PROP_FRAME_COUNT) < nbFrame*dropFrame:
        print("Error in video Reading", videoName, nbFrame)
        print("This video only has ", cap.get(cv2.CAP_PROP_FRAME_COUNT), "frames")
        raise videoFrameError(videoName, nbFrame)
    
    #find possible first frame
    firstFrame = random.randint(0, cap.get(cv2.CAP_PROP_FRAME_COUNT)- (nbFrame*dropFrame))
    
    ret = cap.set(cv2.CAP_PROP_POS_FRAMES,firstFrame) #go to first frame
    cframe = firstFrame #current frame

    if not ret:
        print("Error in video Reading", videoName, cframe)
        print("This video only has ", cap.get(cv2.CAP_PROP_FRAME_COUNT), "frames")
        raise videoFrameError(videoName, cframe)
        
    t = torch.Tensor(nbFrame,3,64,64)
    for i in range(nbFrame):
        ret, frame = cap.read()
        cframe += 1
        if not ret:
            raise videoFrameError(videoName, cframe)    
        t[i] = transform(frame)
    return t.view(t.size(0)*t.size(1), t.size(2), t.size(3))

In [16]:
def VideoDataset(rep="/video/GestureSequence/", SequenceSize=5, batchSize=4, transform=transforms.Compose(
            (transforms.ToPILImage(),
            transforms.Resize(64),
            transforms.RandomCrop(64),
            transforms.ToTensor())), 
            concat=False):
    
    
    if concat:
        openFile = lambda x:ReadNFrame(x,SequenceSize,transform)
    else:
        openFile = lambda x:ReadRandomSequence(x,SequenceSize,transform)
    
    targetOperation = lambda x:int(x)
    
    videodataset = GenericDataset(rep, fileOperation=openFile, targetOperation=targetOperation)
    data_loader = data.DataLoader(videodataset,
                                          batch_size=batchSize,
                                          shuffle=True,
                                          num_workers=4,
                                          drop_last=True
                                )
    return data_loader

In [31]:
def countFrame(fileName):
    cap = cv2.VideoCapture(fileName)
    return cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [32]:
def moveShortVideos(rep="/video/GestureSequence/", trash="/video/VideoTrash"):
    import glob
    for f in glob.iglob(os.path.join(rep,'*')):
        if os.path.isdir(f):
            moveShortVideos(f, trash)
            print("Going into", f)
        else:
            if '.avi' in f:
                print("Handling", f, end='\r')
                nframe = countFrame(f)
                if nframe < 4:
                    print("Moving ", f,end="\t")
                    shutil.move(f, trash)

In [44]:
if __name__=="__main__":
    vd = VideoDataset(rep="/video/GestureSequence/", SequenceSize=3, batchSize=12, concat=True)
    for j,i in enumerate(vd):
        if j==1:
            a = i
        print(j, end="\r")
    print(a)

[51
( 0 , 0 ,.,.) = 
  0.4745  0.4745  0.4706  ...   0.3961  0.2824  0.3333
  0.4784  0.4784  0.4824  ...   0.3647  0.2902  0.3412
  0.4706  0.4745  0.4824  ...   0.3294  0.3098  0.3490
           ...             ⋱             ...          
  0.4078  0.4078  0.4118  ...   0.3255  0.3020  0.3137
  0.4078  0.4039  0.4118  ...   0.3176  0.3059  0.2980
  0.4078  0.4118  0.4118  ...   0.3137  0.3098  0.2902

( 0 , 1 ,.,.) = 
  0.5098  0.5098  0.5020  ...   0.4627  0.3490  0.4039
  0.5098  0.5059  0.5059  ...   0.4353  0.3569  0.4078
  0.5137  0.5137  0.5137  ...   0.4000  0.3765  0.4118
           ...             ⋱             ...          
  0.4863  0.4902  0.4824  ...   0.3961  0.3804  0.3922
  0.4824  0.4863  0.4824  ...   0.3961  0.3843  0.3765
  0.4784  0.4784  0.4784  ...   0.3922  0.3882  0.3725

( 0 , 2 ,.,.) = 
  0.5451  0.5451  0.5412  ...   0.5255  0.4078  0.4588
  0.5490  0.5490  0.5490  ...   0.4941  0.4118  0.4627
  0.5451  0.5451  0.5529  ...   0.4510  0.4235  0.4667
        